In [45]:
%matplotlib widget



import math
import matplotlib.pyplot as plt
import numpy as np
import random
from sklearn.preprocessing import Normalizer
import matplotlib.animation as animation
import plotly
import plotly.graph_objs as go
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from IPython.display import clear_output
from sklearn.linear_model import LogisticRegression
import io
import seaborn as sns; sns.set()
from sklearn import preprocessing
plt.rc("font", size = 14)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
sns.set(style="white")
sns.set(style="whitegrid", color_codes = True)
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb

from mpl_toolkits.mplot3d import Axes3D
# Bundeled Data for visualisation

Object2Mass = 7.3476731e+22
Object2Radius = 1737400 

# Physics Constants
GravConstant =  6.67e-11

# Physics update steps per second and max simulation time
CaptureAmount = 40
CaptureFrequency = 100
CaptureStep = 2
TNOS = 100
MaxSimulationTime = 50000
StepsPerSecond = 1

Presets = input("Type 'd' to select a preset setting or anything else to specify your own. ")

if Presets == "d":
    print("Preset settings:")
    print("Type 't' for TEST settings")
    print("Type 'l' for LOW settings")
    print("Type 'm' for MEDIUM settings")
    print("Type 'h' for HIGH settings")
    
    PresetSettings = input("What preset setting do you choose? ('t', 'l', 'm', or 'h')? ")
    
    if PresetSettings == "t":
        CaptureAmount = 10
        CaptureFrequency = 200
        CaptureStep = 1
        TNOS = 50
        MaxSimulationTime = 5000
        StepsPerSecond = 1
        
    if PresetSettings == "l":
        CaptureAmount = 20
        CaptureFrequency = 200
        CaptureStep = 5
        TNOS = 100
        MaxSimulationTime = 20000
        StepsPerSecond = 1
        
    if PresetSettings == "m":
        CaptureAmount = 60
        CaptureFrequency = 200
        CaptureStep = 2
        TNOS = 200
        MaxSimulationTime = 50000
        StepsPerSecond = 1     
           
    if PresetSettings == "h":
        CaptureAmount = 100
        CaptureFrequency = 150
        CaptureStep = 1
        TNOS = 500
        MaxSimulationTime = 100000
        StepsPerSecond = 1    
else:
    StepsPerSecond = int(input("How many physics simulation steps per second? "))
    TNOS = int(input("How many total simulations would you like to run? "))
    MaxSimulationTime = int(input("How long would you like the simulation to be (in seconds)? "))
    CaptureAmount = int(input("How many saved steps per checkpoint ? "))
    CaptureFrequency = int(input("How often should the step-counting process begin (in seconds)? "))
    CaptureStep = int(input("What should the delay between step measurements be (in seconds)? "))

#MaxSimulationTime = int(input("How long would you like the simulation to be (in seconds)? "))

# Definitions before simulation
velbundley = []
velbundlex = []
velbundlez = []
XCaptureData = []
YCaptureData = []
ZCaptureData = []
TCaptureData = []
CaptureDataframe = []
velbundlet = []
timebundle = []

XBundle = []
YBundle = []
ZBundle = []



# Movement and position updates, the equations are derived from the universal gravitational equation and trigonometry.
NOS = 0
while True:
    clear_output()
    print("Progress: Simulation {} out of {}. {}% done".format(NOS, TNOS, (NOS / TNOS) * 100))
    counter = 0
    TestSimData = []
    SimData = []
    dt = 1 / StepsPerSecond

    # Initial Small Object Conditions
    XPos1 = 1737400 + random.randint(0,200000)
    YPos1 = 1737400 + random.randint(0,200000)
    ZPos1 = 1737400 + random.randint(0,200000)
        

        
    XVel1 = random.randint(-2000,2000)
    YVel1 = random.randint(-2000,2000)
    ZVel1 = random.randint(-2000,2000)

    XPosInital = XPos1
    YPosInital = YPos1
    ZPosInital = ZPos1
    # Large Object Conditions
    XPos2 = 0
    YPos2 = 0
    ZPos2 = 0
    counter = 0
    simulating = True
    t = 0
    while simulating == True:
        # t is the current time, dt is the change in time that is defined by StepsPerSecond
        t = t + dt
        # Tracking positions and velocities of X and Y over time by storing them in lists that are equivelant to the time steps
        
        velbundley.append(YVel1)
        velbundlex.append(XVel1)
        velbundlez.append(ZVel1)
        
        TotalVel = (XVel1 ** 2) + (YVel1 ** 2) + (ZVel1 ** 2)
        
        velbundlet.append(TotalVel)
        timebundle.append(t)
        
        
        XSeperation = (XPos1 - XPos2)
        YSeperation = (YPos1 - YPos2)
        ZSeperation = (ZPos1 - ZPos2)
        
        
        Radius = ((XSeperation ** 2) + (YSeperation ** 2) + (ZSeperation ** 2)) ** 0.5
        
        AccelMag = (GravConstant * Object2Mass) / (Radius ** 2)
        
        
        AngleX = math.acos(XSeperation / Radius)
        AngleY = math.acos(YSeperation / Radius)
        AngleZ = math.acos(ZSeperation / Radius)

        
        XGravAccel1 = -1 * AccelMag * math.cos(AngleX)
        YGravAccel1 = -1 * AccelMag * math.cos(AngleY)
        ZGravAccel1 = -1 * AccelMag * math.cos(AngleZ)
        
        # Angle definition from x and y seperation
        # Angle = math.atan2(YSeperation, XSeperation)
        # XGravAccel1 = -1 * ((GravConstant * Object2Mass) / (Radius ** 2)) * math.cos(Angle)
        # YGravAccel1 = -1 * ((GravConstant * Object2Mass) / (Radius ** 2)) * math.sin(Angle)


        #First measurement
        XPosi = XPos1
        YPosi = YPos1
        ZPosi = ZPos1
        
        XVeli = XVel1
        YVeli = YVel1
        ZVeli = ZVel1

        # Updated Velocity From Acceleration
        XVel1 = XVel1 + XGravAccel1 * dt
        YVel1 = YVel1 + YGravAccel1 * dt
        ZVel1 = ZVel1 + ZGravAccel1 * dt
        # Updated Position From Velocity
        XPos1 = XPos1 + XVel1 * dt
        YPos1 = YPos1 + YVel1 * dt
        ZPos1 = ZPos1 + ZVel1 * dt
        
        XPosf = XPos1
        YPosf = YPos1
        ZPosf = ZPos1
        
        XVelf = XVel1
        YVelf = YVel1
        ZVelf = ZVel1        
        
        if t % CaptureFrequency == 0:
            StopCaptureAt = t + (CaptureAmount * CaptureStep)
            #StoreImportantData = [XPosi, YPosi, ZPosi, XVeli, YVeli, ZVeli, XPosf, YPosf, ZPosf, XVelf, YVelf, ZVelf]
            TakingCapture = 0
            CaptureStart = True
            XCapture = []
            YCapture = []
            ZCapture = []
            TCapture = []
            
            XPosd = XPosf - XPosi
            YPosd = YPosf - YPosi
            ZPosd = ZPosf - ZPosi
            
            XVeld = XVelf - XVeli
            YVeld = YVelf - YVeli
            ZVeld = ZVelf - ZVeli
            
            StoreImportantData = [XPosd, YPosd, ZPosd, XVeld, YVeld, ZVeld]
            SimData.append(StoreImportantData)
            
        Valid = False
        if XPos1 < 4000000 and XPos1 > -4000000:
            if YPos1 < 4000000 and YPos1 > -4000000:
                if ZPos1 < 4000000 and ZPos1 > -4000000:
                    Valid = True
            
        if CaptureStart == True:
            if t % CaptureStep == 0:
                XCapture.append(XPos1)
                YCapture.append(YPos1)
                ZCapture.append(ZPos1)
                if Valid == True:
                    TCapture = XCapture + YCapture + ZCapture
             
            if t == StopCaptureAt:
                XCaptureData.append(XCapture)
                YCaptureData.append(YCapture)
                ZCaptureData.append(ZCapture)  
                if Valid == True:
                    TCaptureData.append(TCapture)
                CaptureStart = False
        
                
                

            
        TestNumberOfSeconds = 0    
        if NOS == TNOS:
            if t % CaptureFrequency == True:
                XBundle.append(XPos1)
                YBundle.append(YPos1)
                ZBundle.append(ZPos1)
                XPosd = XPosf - XPosi
                YPosd = YPosf - YPosi
                ZPosd = ZPosf - ZPosi
                
                XVeld = XVelf - XVeli
                YVeld = YVelf - YVeli
                ZVeld = ZVelf - ZVeli
                StoreImportantData2 = [XPosd, YPosd, ZPosd, XVeld, YVeld, ZVeld]
                
                PositionChain = []
                
                TestSimData.append(StoreImportantData2)    
                TestNumberOfSeconds = TestNumberOfSeconds + 1
                
        counter = counter + 1   
        if t > MaxSimulationTime:
            break
        if Radius < Object2Radius:
            break
    NOS = NOS + 1
    if NOS > TNOS:
        break
print("Simulations done!")    

testfulldataframe = []

fulldataframe = pd.DataFrame(columns=['XPosd', 'YPosd', 'ZPosd', 'XVeld', 'YVeld', 'ZVeld'], data=SimData)

testfulldataframe = pd.DataFrame(columns=['XPosd', 'YPosd', 'ZPosd', 'XVeld', 'YVeld', 'ZVeld'], data=TestSimData)

TestList = []

XDataframe = []
YDataframe = []
ZDataframe = []

PredictionDataFrame = []
CounterPred = 0
PredictionData = []
TestingData = []    
ToPredict = ['XPosd', 'YPosd', 'ZPosd']


XCaptureData = pd.DataFrame(XCaptureData)
YCaptureData = pd.DataFrame(YCaptureData)
ZCaptureData = pd.DataFrame(ZCaptureData)
TCaptureData = pd.DataFrame(TCaptureData)


trainset, testset = train_test_split(TCaptureData, test_size=0.2)

PredictionDataFrame = []
CounterPred = 0
PredictionData = []
TestingData = []
CaptureAmount = CaptureAmount + 1
ToPredict = [CaptureAmount, CaptureAmount * 2, CaptureAmount * 3]
MSEList = []

#LightGBM Optimisation
LGBMReg = lgb.LGBMRegressor(num_iterations = 300, )


while CounterPred < len(ToPredict):
    
    X_train = trainset.drop(trainset.columns[ToPredict[CounterPred] - 1], axis = 1)
    Y_train = trainset.iloc[:,ToPredict[CounterPred] - 1]
    X_test = testset.drop(trainset.columns[ToPredict[CounterPred] - 1], axis = 1)
    Y_test = testset.iloc[:,ToPredict[CounterPred] - 1].copy()
    LGBMReg.fit(X_train, Y_train)
    Y_pred = LGBMReg.predict(X_test)
    LGBMReg.score(X_train, Y_train)
    
    PredictionDataFrame.append(Y_pred)
    
    CounterPred = CounterPred + 1

    MSEList.append(mean_squared_error(Y_test, Y_pred, squared=False))
    print("Step {} of 3 complete.".format(CounterPred))
    
print("Predictions complete! Printing results:")




#XDiff = testfulldataframe['XPosd']
#YDiff = testfulldataframe['YPosd']
#ZDiff = testfulldataframe['ZPosd']

#PredictionDataFrame = pd.DataFrame(PredictionDataFrame)
#PredictionDataFrame = PredictionDataFrame.T
#PredictionDataFrame.columns=['XPosd', 'YPosd', 'ZPosd']



XPosP = XPosInital
YPosP = YPosInital
ZPosP = ZPosInital

XPosPG = []
YPosPG = []
ZPosPG = []
    

#XPosUpdate = PredictionDataFrame['XPosd'].tolist()
#YPosUpdate = PredictionDataFrame['YPosd'].tolist()
#ZPosUpdate = PredictionDataFrame['ZPosd'].tolist()
#AISimTimer = 0
#while AISimTimer < TestNumberOfSeconds:
    
    #XPosP = XPosP + XPosUpdate[AISimTimer]
    #YPosP = YPosP + YPosUpdate[AISimTimer]
    #ZPosP = ZPosP + ZPosUpdate[AISimTimer]
    
    #AISimTimer = AISimTimer + 1
    #XPosPG.append(XPosP)
    #YPosPG.append(YPosP)
    #ZPosPG.append(ZPosP)
    



#drawgraph = int(input("Would you like to draw a graph? 1 for yes, 0 for no: "))
drawgraph = 2
if drawgraph == 1:
    fig = plt.figure(figsize = (7,7))
    ax = fig.add_subplot(111, projection='3d')
    zline = ZBundle
    xline = XBundle
    yline = YBundle
    ax.plot3D(xline, yline, zline, 'red')
    u = np.linspace(0, 2 * np.pi, 100)
    v = np.linspace(0, np.pi, 100)

    XPosPG = ZBundle
    YPosPG = YBundle
    ZPosPG = ZBundle
    
    ax.plot3D(XPosPG, YPosPG, ZPosPG, 'yellow')
    u = np.linspace(0, 2 * np.pi, 100)
    v = np.linspace(0, np.pi, 100)

    ax.set_xlabel('X')
    ax.set_xlim(-Object2Radius * 1, Object2Radius * 1)
    ax.set_ylabel('Y')
    ax.set_ylim(-Object2Radius * 1, Object2Radius * 1)
    ax.set_zlabel('Z')
    ax.set_zlim(-Object2Radius * 1, Object2Radius * 1)

    x = 1737400 * np.outer(np.cos(u), np.sin(v))
    y = 1737400 * np.outer(np.sin(u), np.sin(v))
    z = 1737400 * np.outer(np.ones(np.size(u)), np.cos(v))

    ax.plot_surface(x, y, z, rstride=4, cstride=4, color='b')
    plt.show()


#plt.plot(timebundle, XBundle, label = 'Xx')
#plt.plo    t(timebundle, YBundle, label = 'Yx')
#plt.plot(timebundle, ZBundle, label = 'Zx')
#plt.legend()    
#plt.show()


# Convert everything to functions for n bodies, like calling a method (eg an equation) and using inputs to find it out, eg Calculate(XPosition, YPosition, Weight) (replace with needed numbers) - DONT USE CLASSES
# Combine vectors i n the end and use numpi to store values
#YDataframe.isnull().sum().sum()

pd.options.display.max_columns = 200
pd.options.display.max_rows = 250
TotalDataset = []



# figure size
plt.figure(figsize=(12, 8))
# acutal values
plt.plot([i for i in range(len(Y_test))],Y_test, c='g', label="Real Sim Data")
# predicted values
plt.plot([i for i in range(len(Y_test))],Y_pred, c='m',label="Predicted Values")
plt.legend()
plt.show()
MSEList



Progress: Simulation 1 out of 1. 100.0% done
Simulations done!
